<a href="https://colab.research.google.com/github/mortendahlback/verifiserbar-kart/blob/main/Verifiseringskart_midt%C3%B8sten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importer nødvendige bibliotek
import pandas as pd
import folium
from sklearn.neighbors import KDTree
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Hent data fra Google Sheet og konverter til pandas dataframe
#Importer nødvendige funksjoner og autentiser brukeren din
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

#Åpne regneark og arkfane
worksheet = gc.open('Verifiseringsdatabase')
worksheet = worksheet.worksheet('Midtøsten-kart')

# Hent liste av rader fra arkfanen
rows = worksheet.get_all_values()

#Gjør om til pandas df og konverter koordinater til numerisk format
df = pd.DataFrame.from_records(rows[1:],columns=rows[0])
df[['lat', 'long']] = df[['lat', 'long']].apply(pd.to_numeric)
df.dropna(how='any', inplace=True)

In [ ]:
#Valgfri: Skriv ut kolonner for feilsøking
for col in df.columns:
  print(col)

ID
dato_hendelse
lat
long
maps_url
tittel
oppsummering
bilde_popup
bildelenke


In [ ]:
df.head

In [ ]:
# Bruk KDTree for å finne punktet med høyest tetthet
coordinates = df[['lat', 'long']].values
tree = KDTree(coordinates)

# Finn avstandene til de nærmeste naboene
distances, indices = tree.query(coordinates, k=2)

# Beregn gjennomsnittlig avstand til nærmeste nabo
avg_distances = np.mean(distances[:, 1], axis=0)

# Finn indeksen til punktet med lavest gjennomsnittlig avstand (høyest tetthet)
central_index = np.argmin(avg_distances)

# Bruk koordinatene til dette punktet som sentralpunkt
central_point = coordinates[central_index]

In [ ]:
#Definer en popup-mal med elementer fra celler i regnearket

def create_popup(row):
    html = f"""
    <div style="width: 200px;">
        <h4 style="font-family:Helvetica;">{row['tittel']}</h4>
        <a href="{row['bildelenke']}", target="_blank">
        <img src="{row['bilde_popup']}" style="width:100%;"></a>
        <p style="font-family:Helvetica;">{row['oppsummering']}</p>
        <p style="font-family:Helvetica;"><i>Tidspunkt:</i> {row['dato_hendelse']}</p>
        <a href="{row['maps_url']}", target="_blank">
        <p style="font-family:Helvetica;">Se i Google Maps</p></a>
    </div>
    """
    return folium.Popup(html, max_width=200)

In [ ]:
#Generer kartet
# Opprett kartet i folium med tilesett fra CartoDB
map_file = folium.Map(location=central_point, tiles='CartoDBPositron', zoom_start=11)

# Legg til egendefinert markørikon
icon_path = '/content/drive/Shareddrives/verifisering - Krigen i Israel Gaza/KARTMASKIN/KARTMASKIN Midtøsten/Colab-kart/roundicon.png'

#Generer en markør for hver rad i regnearket
for _, row in df.iterrows():
    popup = create_popup(row)
    icon = folium.features.CustomIcon(icon_path, icon_size=(30, 30))
    folium.Marker([row['lat'], row['long']], popup=popup, icon=icon).add_to(map_file)

# Lagre kartet som en html-fil
map_file_path = '/content/drive/Shareddrives/verifisering - Krigen i Israel Gaza/KARTMASKIN/KARTMASKIN Midtøsten/Colab-kart/midtøsten_kart.html'
map_file.save(map_file_path)